In [1]:
from config import *
from utilities import get_model_name
from dataset import ClassificationDataset

import functools
import numpy as np
import pandas as pd

results_dict = {}
internal, external = 11, 12
encode_method = "dummy"
impute_method = "itard"
fs_method, fs_ratio = "graces", 0.5
norm_method = "maxmin"
classification_dataset = ClassificationDataset(
    internal, external, encode_method, impute_method, fs_method, fs_ratio, norm_method, random_state=SEED)

Number of Selected Features: 100%|██████████| 71/71 [14:26<00:00, 12.20s/it]


In [2]:
# from sklearn.neural_network import MLPClassifier
from utilities import MLPClassifier

model_fn = functools.partial(
    MLPClassifier, random_state=SEED, max_iter=50,
    momentum=0.9, squares=0.999, optimizer_t='sgdm',
    use_residual=True, use_batch_norm=False)
param_grid_list = [{
    # hidden_layer_sizes = (64, 32)
    # config       lr_range            worst_auc
    # adam w/ bn:  [2e-5, 5e-5, 1e-4]  0.8942
    # adam w/o bn: [5e-5, 1e-4, 2e-4]  0.9148
    # sgdm w/ bn:  [1e-3, 2e-3, 5e-3]  0.8982
    # sgdm w/o bn: [5e-3, 1e-2, 2e-2]  0.9177

    'hidden_layer_sizes': [(128, 128), (64, 64, 64)],
    'learning_rate': [5e-3, 1e-2, 2e-2],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

MLPClassifier


model(hidden_layer_sizes=(128, 128), learning_rate=0.005)
Valid:         AUC: 0.9050 | AUPR: 0.6381 | Acc: 95.33%


model(hidden_layer_sizes=(128, 128), learning_rate=0.01)
Valid:         AUC: 0.9100 | AUPR: 0.6455 | Acc: 95.30%


model(hidden_layer_sizes=(128, 128), learning_rate=0.02)
Valid:         AUC: 0.9128 | AUPR: 0.6503 | Acc: 95.23%


model(hidden_layer_sizes=(64, 64, 64), learning_rate=0.005)
Valid:         AUC: 0.9072 | AUPR: 0.6379 | Acc: 95.29%


model(hidden_layer_sizes=(64, 64, 64), learning_rate=0.01)
Valid:         AUC: 0.9112 | AUPR: 0.6449 | Acc: 95.27%


model(hidden_layer_sizes=(64, 64, 64), learning_rate=0.02)
Valid:         AUC: 0.9127 | AUPR: 0.6472 | Acc: 95.27%
best model(hidden_layer_sizes=(128, 128), learning_rate=0.02)
Valid:         AUC: 0.9128 | AUPR: 0.6503 | Acc: 95.23%
Internal test: AUC: 0.9203 | AUPR: 0.6745 | Acc: 94.77%
External test: AUC: 0.9137 | AUPR: 0.6545 | Acc: 95.16%


In [3]:
from sklearn.svm import SVC

model_fn = functools.partial(
    SVC, random_state=SEED, probability=True,
    class_weight='balanced')
param_grid_list = [{
    'kernel': ['linear'],
    'C': [0.01, 0.1, 1],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

SVC
model(kernel=linear, C=0.01)
Valid:         AUC: 0.9090 | AUPR: 0.5977 | Acc: 94.78%
model(kernel=linear, C=0.1)
Valid:         AUC: 0.9101 | AUPR: 0.5955 | Acc: 94.82%
model(kernel=linear, C=1)
Valid:         AUC: 0.9102 | AUPR: 0.5977 | Acc: 94.88%
best model(kernel=linear, C=1)
Valid:         AUC: 0.9102 | AUPR: 0.5977 | Acc: 94.88%
Internal test: AUC: 0.9200 | AUPR: 0.6450 | Acc: 94.82%
External test: AUC: 0.9075 | AUPR: 0.5943 | Acc: 94.61%


In [4]:
from sklearn.neighbors import KNeighborsClassifier

model_fn = functools.partial(
    KNeighborsClassifier, n_jobs=-1, algorithm='auto', p=2)
param_grid_list = [{
    'n_neighbors': [3, 5, 7],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

KNeighborsClassifier
model(n_neighbors=3)
Valid:         AUC: 0.7486 | AUPR: 0.4063 | Acc: 94.62%
model(n_neighbors=5)
Valid:         AUC: 0.7728 | AUPR: 0.4513 | Acc: 94.88%
model(n_neighbors=7)
Valid:         AUC: 0.7905 | AUPR: 0.4806 | Acc: 94.90%
best model(n_neighbors=7)
Valid:         AUC: 0.7905 | AUPR: 0.4806 | Acc: 94.90%
Internal test: AUC: 0.7838 | AUPR: 0.4809 | Acc: 94.60%
External test: AUC: 0.7928 | AUPR: 0.4775 | Acc: 94.62%


In [5]:
from sklearn.naive_bayes import GaussianNB

model_fn = GaussianNB
param_grid_list = [{}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

GaussianNB
model()
Valid:         AUC: 0.8709 | AUPR: 0.4289 | Acc: 82.16%
best model()
Valid:         AUC: 0.8709 | AUPR: 0.4289 | Acc: 82.16%
Internal test: AUC: 0.8791 | AUPR: 0.4439 | Acc: 82.00%
External test: AUC: 0.8699 | AUPR: 0.4099 | Acc: 81.37%


In [6]:
# from sklearn.exceptions import ConvergenceWarning
# import warnings
# warnings.filterwarnings("ignore", category=ConvergenceWarning)
from sklearn.linear_model import LogisticRegression

model_fn = functools.partial(
    LogisticRegression, random_state=SEED, n_jobs=-1, 
    max_iter=2000, solver='saga', penalty='l2',
    )
param_grid_list = [{
    'C': [1.0, 10.0, 100.0],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(
    model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(
    best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

LogisticRegression
model(C=1.0)
Valid:         AUC: 0.9104 | AUPR: 0.6410 | Acc: 95.30%
model(C=10.0)
Valid:         AUC: 0.9103 | AUPR: 0.6408 | Acc: 95.26%
model(C=100.0)
Valid:         AUC: 0.9102 | AUPR: 0.6407 | Acc: 95.25%
best model(C=1.0)
Valid:         AUC: 0.9104 | AUPR: 0.6410 | Acc: 95.30%
Internal test: AUC: 0.9207 | AUPR: 0.6687 | Acc: 95.18%
External test: AUC: 0.9080 | AUPR: 0.6303 | Acc: 95.08%


In [7]:
from sklearn.tree import DecisionTreeClassifier

model_fn = functools.partial(
    DecisionTreeClassifier, random_state=SEED,
    ccp_alpha=0.0, class_weight='balanced', max_depth=5)
param_grid_list = [{
        'criterion': ['gini', 'log_loss', 'entropy'],
    }
]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

DecisionTreeClassifier
model(criterion=gini)
Valid:         AUC: 0.8548 | AUPR: 0.5477 | Acc: 85.47%
model(criterion=log_loss)
Valid:         AUC: 0.8562 | AUPR: 0.5649 | Acc: 84.17%
model(criterion=entropy)
Valid:         AUC: 0.8562 | AUPR: 0.5649 | Acc: 84.17%
best model(criterion=log_loss)
Valid:         AUC: 0.8562 | AUPR: 0.5649 | Acc: 84.17%
Internal test: AUC: 0.8670 | AUPR: 0.6063 | Acc: 85.21%
External test: AUC: 0.8627 | AUPR: 0.5772 | Acc: 84.49%


In [8]:
from sklearn.ensemble import RandomForestClassifier

model_fn = functools.partial(
    RandomForestClassifier, random_state=SEED, n_jobs=-1,
    ccp_alpha=0.0, class_weight='balanced', max_depth=5, bootstrap=False)
param_grid_list = [{
        'criterion': ['gini', 'log_loss', 'entropy'],
        'n_estimators': [50, 100, 200],
    }
]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

RandomForestClassifier
model(criterion=gini, n_estimators=50)
Valid:         AUC: 0.9059 | AUPR: 0.6426 | Acc: 89.51%
model(criterion=gini, n_estimators=100)
Valid:         AUC: 0.9067 | AUPR: 0.6446 | Acc: 89.59%
model(criterion=gini, n_estimators=200)
Valid:         AUC: 0.9065 | AUPR: 0.6438 | Acc: 89.77%
model(criterion=log_loss, n_estimators=50)
Valid:         AUC: 0.9065 | AUPR: 0.6453 | Acc: 89.77%
model(criterion=log_loss, n_estimators=100)
Valid:         AUC: 0.9067 | AUPR: 0.6457 | Acc: 89.83%
model(criterion=log_loss, n_estimators=200)
Valid:         AUC: 0.9068 | AUPR: 0.6459 | Acc: 89.98%
model(criterion=entropy, n_estimators=50)
Valid:         AUC: 0.9065 | AUPR: 0.6453 | Acc: 89.77%
model(criterion=entropy, n_estimators=100)
Valid:         AUC: 0.9067 | AUPR: 0.6457 | Acc: 89.83%
model(criterion=entropy, n_estimators=200)
Valid:         AUC: 0.9068 | AUPR: 0.6459 | Acc: 89.98%
best model(criterion=log_loss, n_estimators=200)
Valid:         AUC: 0.9068 | AUPR: 0.6459 | Ac

In [9]:
from sklearn.ensemble import GradientBoostingClassifier

model = functools.partial(
    GradientBoostingClassifier, random_state=SEED,
    learning_rate=0.1, max_depth=5, loss='log_loss', 
    n_estimators=100)
param_grid_list = [{
        'criterion': ['friedman_mse', 'squared_error'],
    }
]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

GradientBoostingClassifier
model(criterion=friedman_mse)
Valid:         AUC: 0.9123 | AUPR: 0.6366 | Acc: 95.25%
model(criterion=squared_error)
Valid:         AUC: 0.9125 | AUPR: 0.6366 | Acc: 95.21%
best model(criterion=squared_error)
Valid:         AUC: 0.9125 | AUPR: 0.6366 | Acc: 95.21%
Internal test: AUC: 0.9196 | AUPR: 0.6799 | Acc: 95.04%
External test: AUC: 0.9109 | AUPR: 0.6490 | Acc: 95.13%


In [10]:
from sklearn.ensemble import AdaBoostClassifier
model = functools.partial(AdaBoostClassifier, random_state=SEED, algorithm="SAMME")
param_grid_list = [{
    'n_estimators': [100, 200, 500],
    'learning_rate': [1.0],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

AdaBoostClassifier
model(n_estimators=100, learning_rate=1.0)
Valid:         AUC: 0.9102 | AUPR: 0.6386 | Acc: 95.33%
model(n_estimators=200, learning_rate=1.0)
Valid:         AUC: 0.9104 | AUPR: 0.6381 | Acc: 95.32%
model(n_estimators=500, learning_rate=1.0)
Valid:         AUC: 0.9106 | AUPR: 0.6382 | Acc: 95.32%
best model(n_estimators=500, learning_rate=1.0)
Valid:         AUC: 0.9106 | AUPR: 0.6382 | Acc: 95.32%
Internal test: AUC: 0.9172 | AUPR: 0.6627 | Acc: 95.16%
External test: AUC: 0.9084 | AUPR: 0.6323 | Acc: 95.13%


In [11]:
dataframe = pd.DataFrame(results_dict).transpose().reset_index()
dataframe.columns = ['model', 'val_auc', 'val_aupr', 'val_acc', 'intest_auc', 'intest_aupr', 'intest_acc', 'extest_auc', 'extest_aupr', 'extest_acc']
# dataframe.to_csv()
filename = os.path.join(RESULTS_DIR, encode_method, impute_method, f'{fs_method}{int(fs_ratio * 142)}.csv')
dirname = os.path.dirname(filename)
if not os.path.exists(dirname):
    os.makedirs(dirname)
dataframe.to_csv(filename, index=False)